In [5]:
import sys
import time

sys.path.append('../../..')

In [6]:
import tiktoken

from omegaconf import OmegaConf
from openai import OpenAI
from tqdm import tqdm

from src.data_prepocessing import load_ds
from src.evaluation import Evaluator

# Experiment set up

In [7]:
config = OmegaConf.load("openai_config.yaml")
print(OmegaConf.to_yaml(config))

model_name: gpt-4o-mini
prompt: Ты професиональный разметчик с лингвистическим образованием. Внимательно прочитай
  диалог. Перепишите неполное высказывание на основе истории диалога. Ответ должен
  содержать только переписанное высказывание



# Data loading

In [8]:
class HFCompatibleTokenizer:
    def __init__(self, model_name):
        self._tokenizer = tiktoken.encoding_for_model(model_name)

    def encode(self, text):
        return {'input_ids': self.tokenizer.encode(text)}
    
    def __call__(self, text):
        return self.encode(text)
    

tokenizer = HFCompatibleTokenizer(config.model_name)

In [ ]:
ds = load_ds("2rca_checked_version.json")

Map: 100%|██████████| 551/551 [00:00<00:00, 4107.86 examples/s]


In [ ]:
ds['train'][0]

{'history': ['Привет.',
  'Привет.',
  'Как твоё настроение сегодня?',
  'Настроение отлично, скоро куплю машину, а твоё настроение как?',
  'Здорово, ты скоро купишь машину! У меня тоже настроение на позитиве! Я очень давно хочу в Бразилию, и, кажется, сегодня моя мечта сбудется! Вечером поедем с подругой в турфирму оплачивать наш тур в Бразилию. А у тебя есть мечта?',
  'Давно хочу завести лабрадора, прям с детства хочу собаку. Может, когда дочка подрастет заведу себе лабрадора.',
  'Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин?'],
 'phrase': 'Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.',
 'rewrite': 'Сегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'}

# Evaluation

In [ ]:
def infer_ds(ds, model, **kwargs):
    test_results = []

    for i in tqdm(range(100)):
        success = False
        while not success:
            try:
                messages = [
                    {
                        "role": "system",
                        "content": config.prompt
                    },

                    {
                        "role": "user",
                        "content": "История: " + str(ds['train']['history'][0][-1]) + " Неполное высказвание: " + ds['train']["phrase"][0]
                    },
                    {
                        "role": "assistant",
                        "content": ds['train']['rewrite'][0]
                    },

                    {
                        "role": "user",
                        "content": "История: " + str(ds['train']['history'][1][-1]) + " Неполное высказвание: " + ds['train']["phrase"][1]
                    },
                    {
                        "role": "assistant",
                        "content": ds['train']['rewrite'][1]
                    },

                    {
                        "role": "user",
                        "content": "История: " + str(ds['train']['history'][2][-1]) + " Неполное высказвание: " + ds['train']["phrase"][2]
                    },
                    {
                        "role": "assistant",
                        "content": ds['train']['rewrite'][2]
                    },


                    {
                        "role": "user",
                        "content": "История: " + str(ds['test']['history'][i][-1]) + " Неполное высказвание: " + ds['test']["phrase"][i]
                    }
                ]

                response = kwargs["client"].chat.completions.create(
                    model=model,
                    messages=messages,
                    temperature=1
                )
                out = response.choices[0].message.content
                test_results.append(out)
                success = True
            except:
                time.sleep(20)

    return test_results

In [ ]:
client = OpenAI()

evaluator = Evaluator(dataset=ds, 
                      model=config.model_name, 
                      client=client,
                      tokenizer=tokenizer, 
                      infer_func=infer_ds)

evaluator.evaluate()

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,63.625389,0.643337,0.487672,0.388371,0.307908,0.63482,0.396367,0.304423,0.26781,0.246996
